# Accessibility to health facilities

This is an exploratory notebook that leverages Partnership data. We are guided by the suggestions and pointers given in our meetings on the subjects with experts across the Bank and external partners.

## Setup

If you are running ddp code for the first time, you need to install the library that includes many helper functions to easy access to partner data.

You need to be a added to the [Github repository](https://github.com/datapartnership/devdatapartnership) to get access. If you cannot see it, you don't have access to install the library. If so, please contact datapartnership@worldbank.org with your Github username to get added to the repository.

```sh
conda create -n ddp python=3
conda activate ddp
/path/to/python -m pip install -e git+ssh://git@github.com/datapartnership/devdatapartnership.git#egg=devdatapartnership
```
Note: You need to restart the kernel to be able to import after install

In [ ]:
#import sys ; sys.executable

### Authenticate using DDP credentials


In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
env_path = Path('~/')/'.ddp' / '.env'
load_dotenv(dotenv_path=env_path)
load_dotenv(override=True);
MAPBOX_ACCESS_TOKEN = os.getenv("MAPBOX_ACCESS_TOKEN")

location='Senegal'
data_folder=Path('.')/'data'/location


## Accesibility to health facilities

This section explores who to measure accesibility, in minutes, to health facilities. 

We will assume that a region needs to know how far away (by car) their entire population is from their closest hospital. You can use this to see regions that are too far away, or where to locate additional services that cater those most in need.

### Routing engine

One of our partners, Mapbox, offers traffic-aware global travel times. We will set up the functions to calculate distance in minutes walking, biking, driving and driving with real-time traffic.

In [ ]:
#Simple point to point call

import requests  #http framework to make Mapbox API requests for routes
import json # handle response as json
import datetime # save timestamp

osrm_server="https://api.mapbox.com/directions-matrix/v1/mapbox/"
modes=['driving-traffic', 'driving', 'cycling', 'walking']
mode = modes[1]
url=osrm_server+mode+'/'
params="?annotations=distance,duration&access_token="+MAPBOX_ACCESS_TOKEN
comma="%2C"
sep="%3B"

origin=[43.394020,-5.706718]
destination=[43.523757,-6.047233]
fullurl=url+str(origin[1])+','+str(origin[0])+";"+str(destination[1])+','+str(destination[0])+params
response = requests.get(fullurl) #do the request
response.raise_for_status() # ensure we notice bad responses
response=json.loads(response.text)
#print(response)
print, fullurl
print("Between La Corrada and Noreña there are %2.1f km and ~%2.0f minutes driving."%
      (response['distances'][0][1]/1000.,
       response['durations'][0][1]/60.))

### Data we need

1. Origins: From where to start to travel time (population centers, zip codes, cities, ...)
2. Population on origins: How many people are located at the origin?
3. Destinations: The set of destinations. Health facilities in this case

In our case, as a proxy for 1. and 2. we will use our partner Facebook and their [population density map](https://data.humdata.org/dataset/highresolutionpopulationdensitymaps). We also have estimates by age bracket at lower resoltion from [worldpop](https://www.worldpop.org/project/categories?id=8). We might use these on a later stage to estimate impact.

For 3 we use the hospitals and health centers tagged as such on OSM (using e.g. [overpass-turbo](https://overpass-turbo.eu/s/Rlq))

We choose Senegal, as they are one of the developing countries with highest known cases


#### Population density

Population Density maps are **huge** files where every pixel is ~23m^2. To manage the computation our strategy is to first cluster the country in windows of size e.g. `2000 pixels`. If the window has no population inside, we skip. If it does be split the window in 4 sub-windows, and repeat the process until the window has an arbitraty minimum size e.g. `125 pixels`.

In [ ]:
pop_density_url="https://data.humdata.org/dataset/dbd7b22d-7426-4eb0-b3c4-faa29a87f44b/resource/f84782aa-b726-4370-91cc-9bca318ee067/download/population_sen_2018-10-01.zip"

In [ ]:
#Get the data
import urllib

print("Getting %s population data"%(location) )

if not os.path.exists(data_folder):
    os.makedirs(data_folder)

pop_file=data_folder/'pop_density/'
if not os.path.exists(pop_file.with_suffix('.zip')):
    print("Downloading pop density")
    urllib.request.urlretrieve (pop_density_url,pop_file.with_suffix('.zip'))
if not os.path.exists(pop_file):
    print("Unzipping pop density")
    urllib.request.urlretrieve (pop_density_url,pop_file)

#TODO get this file
map_file=pop_file/'population_sen_2018-10-01.tif'
print("Map file: %s"%map_file)

In [ ]:
import numpy as np
import rasterio
import pandas as pd
from rasterio.windows import Window
from matplotlib import pyplot


def get_pop(map_file,left_x,top_y,window,plot=False):
    """
    get_pop(raster filename,left_x,top_y,window,plot=False)
    
    Given a raster file, and row,cols ranges,
    return the lonlat of the ranges, nancount, and the nunsum
    
    Optionally plot the raster window [False]
    """
    right_x,bottom_y = left_x + window, top_y + window
    with rasterio.open(map_file) as src:
        left_lon, top_lat = src.xy(top_y,left_x )
        right_lon, bottom_lat = src.xy(bottom_y, right_x )
        center_lon , center_lat = (right_lon + left_lon)/2., (top_lat+bottom_lat)/2.
                             #Window(col_off, row_off, width, height)
        w = src.read(1, window=Window(left_x, top_y, window, window))
        if plot:
            pyplot.imshow(w, cmap='pink')
            pyplot.show()
        nancount=np.count_nonzero(~np.isnan(w))
        count = np.size(w)
        tot_pop=np.nansum(w)
    if count == 0:
        return {} #Out of bounds
    if tot_pop == 0 or window < 1: #Mark the window to furhter split.
        split=False
    else:
        split=True
    out={'window':window,
         'left_x':left_x,
         'right_x':right_x,
         'top_y':top_y,
         'bottom_y':bottom_y,
         'left_lon':left_lon, 
         'top_lat':top_lat, 
         'right_lon':right_lon,
         'bottom_lat':bottom_lat,
         'center_lon':center_lon , 
         'center_lat':center_lat,
         'count': count,
         'nancount':nancount,
         'tot_pop':tot_pop,
         'split': split}
    return out

In [ ]:
#Scan the raster map with big windows
origins=pd.DataFrame()
window=2000
with rasterio.open(map_file) as src:
    for left_x in np.arange(0,src.width,window):
        for top_y in np.arange(0,src.height,window):
            out=get_pop(map_file,left_x,top_y,window,plot=False)
            if out != {}:
                origins=origins.append([out])
        print("%i/%i\r"%(left_x,src.width),end="")

In [ ]:
def split(map_file,origin,plot=False):
    """
    Split a window row in 4 parts, and return new rows results
    """
    origins=pd.DataFrame()
    
    window=int(origin.window/2)
    for left_x in np.arange(origin.left_x,origin.right_x,window):
        for top_y in np.arange(origin.top_y,origin.bottom_y,window):
            out=get_pop(map_file,left_x,top_y,window,plot=plot)
            if out != {}:
                origins=origins.append([out])
    return origins

In [ ]:
#Do a splitting pass
#run this cell as many times as you want to split the windows
print("%i regions need splitting"%len(origins[origins['split']==True]))
olen=len(origins)
for i in np.arange(olen):
    print("%i/%i\r"%(i+1,olen),end="")
    if origins.iloc[i,origins.columns.get_loc('split')] == True:
        origins.iloc[i,origins.columns.get_loc('split')]='done'
        s=split(map_file,origins.iloc[i])
        origins=origins.append(s,sort=False)
print("done.")
print("We now have %i regions, %i will be split in next round"%(len(origins),len(origins[origins['split']==True])))

In [ ]:
origins=origins[origins['tot_pop']>0]
origins=origins[origins['split']!='done']
print("We have %i regions of size %i, %i with population >0"%
      (len(origins),min(origins['window']),len(origins[origins['tot_pop']>0])))

In [ ]:
#make it geopandas using the center latlon as location
import geopandas as gdp
from shapely.geometry import Point
origins=gdp.GeoDataFrame(origins,crs='epsg:4327', geometry=[Point(xy) for xy in zip(origins['center_lon'], origins['center_lat'])])
origins.head(2)

In [ ]:
len(origins['window'])

#### Hospital destination

In [ ]:
hospitals_file=data_folder/'hospitals.geojson'
print("Destination file: %s"%hospitals_file)

In [ ]:
import contextily as ctx
import geopandas as gpd

hospitals = gpd.read_file(hospitals_file).to_crs('epsg:3857')

In [ ]:
hospitals['lat']=hospitals.to_crs("epsg:4327").geometry.y
hospitals['lon']=hospitals.to_crs("epsg:4327").geometry.x

ax = hospitals.plot(figsize=(10, 10), alpha=0.8,color='red')
ctx.add_basemap(ax)
ax.set_axis_off()
print("There are %i hospital destinations"%len(hospitals))

### Gather travel times

In [ ]:
def n_closest_geodetic(destinations,origins,n_keep,verbose=False):
    """
    Given a list of origins and destinations, return the "keep" number
    of destinations that are closest geodetically to each origin.
    
    Input: destinations,origins <Geopandas>
    Output: destinations filtered <Geopandas>
    """
    destinations=destinations.to_crs(origins.crs)
    filtered=gdp.GeoDataFrame()
    if verbose:
        i=0
        l=len(origins.index)
    for index in origins.index:
        if verbose:
            i=i+1
            print("Doing %i of %i\r"%(i,l),end="")
        distances=destinations.distance(origins.loc[index].geometry)
        if len(distances) < n_keep:
            n_keep = len(distances)
        #query indices
        indices=np.argsort(distances.values)[:n_keep]
        values=np.sort(distances.values)[:n_keep]
        #destination indices
        d_indices=distances.index[indices]
        filtered = filtered.append(destinations.iloc[indices])
    if verbose:
        print('done')
    return filtered.append(filtered).drop_duplicates(inplace=False)


In [ ]:
test_o = origins.sample(frac=1).reset_index(drop=True).head(10)
test_d = hospitals.sample(frac=1).reset_index(drop=True).head(10)
filtered = n_closest_geodetic(test_d,test_o,2,verbose=False)

In [ ]:
import contextily as ctx
ax = test_d.to_crs('epsg:3857').plot(figsize=(10, 10), alpha=0.8,color='red')
filtered.to_crs('epsg:3857').plot(figsize=(10, 10), alpha=0.8,color='yellow',marker='+',ax=ax)
test_o.to_crs('epsg:3857').plot(figsize=(10, 10), alpha=0.8,color='blue',marker='*',ax=ax)
ctx.add_basemap(ax,zoom=7)
ax.set_axis_off()

In [ ]:
#reset index to be incerasing integers

h=hospitals.reset_index(drop=True)
o=origins.reset_index(drop=True)

In [ ]:
import pickle
#SAVE
with open('h-o52k.pickle', 'wb') as handle:
    pickle.dump([h,o], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('h-o52k.pickle', 'rb') as handle:
    h,o = pickle.load(handle)

In [ ]:
import pandas as pd 
import numpy as np
import geopandas as gdp
def mapbox_matrix_API(origins, destinations, mode=1, name='poi',n_keep=5,do_all=False,verbose=True):
    """
    Given a geopandas set of origins and destinations, return the origins with extra columns
    with the closest destination in minutes given the mode of transportation for each origin.
    
    Also returns the snap distance to the origin (geodetic distance from origin point to closest road)
    Keywords:
    do_all [False]: By default avoid repeating work that has been done.
    
    """
    osrm_server="https://api.mapbox.com/directions-matrix/v1/mapbox/"
    modes=['driving-traffic', 'driving', 'cycling', 'walking']
    url=osrm_server+modes[mode]+'/'
    params="?annotations=distance,duration&access_token="+MAPBOX_ACCESS_TOKEN


    o_type=name
    buffer=10/60.  #10 minutes, in hours
    overalpenalty=1.05  #5%
    batch=int(25/n_keep)
    

    if n_keep*batch>25:
        print("limit 25< %i (keep) * %i (batch)"%(n_keep,batch))

    cols=['t_'+o_type,'m_'+o_type,'so_'+o_type]
    
    #only do empty ones
    if not do_all:
        queued_origins=origins[origins['t_hospital']==-1]
    else:
        queued_origins=origins
    if verbose:
        print("Doing %i"%len(queued_origins))
    try:
        for i in np.arange(queued_origins.shape[0]/batch):
            print("Doing batch %i, from %i to %i, of %i"
              %(i+1,batch*i,batch*(i+1),queued_origins.shape[0]),end="\r")
            #get origin batch
            o_batch=queued_origins.iloc[int(batch*i):].head(n=batch)


            #to reduce API calls calculate keep only n closest (geodetically) to
            #each origin.
            h_batch=n_closest_geodetic(destinations,o_batch,n_keep)
            h_batch_loc=";".join([str(i[1])+','+str(i[0]) for i in h_batch[['lat','lon']].values])

            #create url params of origin batch
            d=";".join([str(i[1])+','+str(i[0]) for i in o_batch[['center_lat','center_lon']].values])
            d_name=o_batch.index

            trail=".json?destinations="+\
            ';'.join([str(x) for x in np.arange(len(h_batch))])+\
            "&sources="+\
            ';'.join([str(x) for x in np.arange(len(h_batch),len(h_batch)+len(o_batch))])

            fullurl= url+h_batch_loc+";"+d+trail+params

            #print(fullurl)
            response = requests.get(fullurl)
            response.raise_for_status()
            response=json.loads(response.text)
            #print(response)
            response_matrix=response['durations']
            durations=[]
            h_min=[]
            for origin in np.arange(np.shape(response_matrix)[0]):
                durations+=[min(response_matrix[origin])]
                h_min+=[np.argmin(response_matrix[origin])]
            for i in np.arange(len(durations)):
                queued_origins.loc[[d_name[i]], ['t_'+o_type]]=buffer+durations[i]/60./60.*overalpenalty
                queued_origins.loc[[d_name[i]], ['m_'+o_type]]=h_min[i]
                queued_origins.loc[[d_name[i]], ['so_'+o_type]]=response['sources'][i]['distance']
    except Exception as err:
        print("Error. Work done has been saved and returned. \nError:",err)
    print("\n")
    #update the "origins" with the results
    
    pd.set_option('mode.chained_assignment', None) #'warn'
    origins.loc[queued_origins.index, (cols)] = queued_origins.loc[:,(cols)].copy()
    print("returning")
    
    return origins


In [ ]:
#initialize columns
o_type = 'hospital'
cols=['t_'+o_type,'m_'+o_type,'so_'+o_type]
for col in cols:
    o[col]=-1

In [ ]:
print(len(o[o['t_hospital']==-1]))

In [ ]:
o = mapbox_matrix_API(o, h, name='hospital',n_keep=5,verbose=True)

In [ ]:
o.to_file("origins", driver='Shapefile')

In [ ]:
h.to_file("h.geojson", driver='GeoJSON')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
o['t_'+o_type].plot.hist(alpha=0.5,bins=100,cumulative=False,density=True,log=True,weights=o['tot_pop'])
plt.ylabel('Population %')
plt.xlabel('Distance to closest: '+o_type+' [h]')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
o['t_'+o_type].plot.hist(alpha=0.5,bins=100,cumulative=False,density=True,log=True)
plt.axvline(5,color="red")
plt.ylabel('Population %')
plt.xlabel('Distance to closest: '+o_type+' [h]')
plt.show()

In [ ]:
threshold=5

o_above = o[o['t_'+o_type]>threshold]
print("%i people (%.2f%%) live beyond %i hours driving from a hospital"%\
     (o_above['tot_pop'].sum(),o_above['tot_pop'].sum()/o['tot_pop'].sum()*100,threshold))

In [ ]:
import contextily as ctx
ax = o_above.to_crs('epsg:3857').plot(figsize=(10, 10), alpha=0.8,color='red')
h.to_crs('epsg:3857').plot(figsize=(10, 10), alpha=0.8,color='blue',ax=ax)
ctx.add_basemap(ax,zoom=7)
ax.set_axis_off()

In [ ]:
from matplotlib import gridspec
import contextily as ctx

def biplot(threshold):
    o_above = o[o['t_'+o_type]>threshold]
    
    fig = plt.figure(figsize=(12, 6)) 
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 

    ax1=plt.subplot(gs[0])

    o_above.to_crs('epsg:3857').plot( alpha=0.8,color='red',ax=ax1)
    h.to_crs('epsg:3857').plot( alpha=0.8,color='blue',ax=ax1)
    
    ctx.add_basemap(ax1,zoom=7)
    ax1.set_title("(Red) population beyond %i h from hospital (Blue)"%threshold)
    ax1.set_axis_off()
    
    ax2=plt.subplot(gs[1])
    o['t_'+o_type].plot.hist(alpha=0.5,bins=100,cumulative=False,density=True,log=True,ax=ax2)
    ax2.axvline(5,color="red")
    ax2.set_ylabel('Population %')
    ax2.set_xlabel('Distance to closest: '+o_type+' [h]')
    ax2.set_title("%i people (%.2f%%) beyond %i h driving closest hospital"%\
     (o_above['tot_pop'].sum(),o_above['tot_pop'].sum()/o['tot_pop'].sum()*100,threshold))

    plt.tight_layout()
    plt.savefig('Senegal-threshold-%i.pdf'%threshold)
    plt.show();

In [ ]:
biplot(20)

In [ ]:
biplot(5)

In [ ]:
biplot(2)